In [1]:
import pandas as pd
from tqdm import tqdm
from lightning.fabric import Fabric
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import sys
sys.path.append('..')
sys.path.append('../models')

import Effe_CNN1d
import utils

In [2]:
model_hub = ['timm/tf_efficientnet_b0.in1k',
             'efficientnet_b5',]
kernel_size = [3, 5, 7, 17, 19, 21]

In [3]:
df = pd.read_csv(f'../hms-train/train.csv')
train_df, val_df = train_test_split(df, test_size=0.2, random_state=8056)

train_dataset = utils.EEGCleanDataset(train_df)
val_dataset = utils.EEGCleanDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model = Effe_CNN1d.Effe_CNN1d(model_hub[0], kernel_size)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
kl_loss_fn = nn.KLDivLoss(reduction='batchmean')

# fabric module
fabric = Fabric(accelerator='cuda', precision='16-mixed')
model, optimizer = fabric.setup(model, optimizer)
train_loader = fabric.setup_dataloaders(train_loader)
val_loader = fabric.setup_dataloaders(val_loader)

Using 16-bit Automatic Mixed Precision (AMP)


In [4]:
model

_FabricModule(
  (_forward_module): Effe_CNN1d(
    (parrallel_cnn1d): ModuleList(
      (0): Conv1d(16, 64, kernel_size=(3,), stride=(1,), padding=same)
      (1): Conv1d(16, 64, kernel_size=(5,), stride=(1,), padding=same)
      (2): Conv1d(16, 64, kernel_size=(7,), stride=(1,), padding=same)
      (3): Conv1d(16, 64, kernel_size=(17,), stride=(1,), padding=same)
      (4): Conv1d(16, 64, kernel_size=(19,), stride=(1,), padding=same)
      (5): Conv1d(16, 64, kernel_size=(21,), stride=(1,), padding=same)
    )
    (avgpool): AvgPool1d(kernel_size=(10,), stride=(10,), padding=(0,))
    (batchnorm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
    (model): EfficientNet(
      (conv_stem): Conv2dSame(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inpl